## Imports and settings

In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../src/data/')
import util as util

In [7]:
setup, dset = util.read_setup_and_data()
validated = util.validate_setup_and_data(setup, dset)

if not validated:
    print("ERROR: setup and data files are not valid.")
    raise KeyboardInterrupt

# replace dset columns with setup keys
dset.columns = setup.keys()

# # process checkbox responses
dset = util.process_checkboxes(dset, setup)

# save the processed dataset
util.save_data(dset)


# remove respondents who did not agree to share data
nrsp_raw = dset.shape[0]
dset = dset[dset.q1 == "Yes"]
nrsp = dset.shape[0]

print(f"Questions  : {dset.shape[1]}")
print(f"Respondents - all           : {nrsp_raw}")
print(f"Respondents - agreed sharing: {nrsp}")


Questions  : 37
Respondents - all           : 36
Respondents - agreed sharing: 36
